In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os.path import join
folder = join('..', 'data', 'fasttext')

In [3]:
import fastText
import numpy as np
import re
from sklearn.neighbors import NearestNeighbors
import sklearn
import pymorphy2
import tqdm
import random
import os
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import warnings
warnings.filterwarnings('ignore')


morph = pymorphy2.MorphAnalyzer()

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
# очищает текст от знаков преминания
# возвращает массив из слов текста в нормальной форме

def process_text(text, dict_verbs={}):
    s = "".join(text)
    s = re.sub(r"[^A-Za-zА-Яа-я]+", " ", s)
    words = [word.lower() for word in s.split(" ")]   
    words_ = []
    for word in tqdm.tqdm_notebook(words):
        words_ += [morph.parse(word)[0].normal_form]
        part_of_sp = str(morph.parse(word)[0].tag).split(",")[0]
        if part_of_sp in ['VERB', 'INFN']:
            if words_[-1] in dict_verbs.keys():
                dict_verbs[words_[-1]].add(word)
            else:
                dict_verbs[words_[-1]] = set([word])
    return words_

In [5]:
texts = []
for name in os.listdir(folder + '\\train'):
    print(name)
    texts += open(join(folder + '\\train', name))

О комитете информационных технологий и связи ЕАО.txt
О комитете образования ЕАО.txt
О комитете по физической культуре и спорту ЕАО.txt
О комитете социальной защиты населения ЕАО.txt
О контрольно-счетной палате ЕАО.txt
О полномочном представителе Президента РФ.txt
ПП О Минздрав.txt
ПП О Минпромторг.txt
ПП О Минсельхоз.txt
ПП О Минтраспорта.txt


In [6]:
words = process_text(texts)

In [7]:
s = " ".join(words)
tmp = open('tmp.txt', mode='w')
tmp.write(s)

405153

In [ ]:
#model = fastText.train_unsupervised('tmp.txt', dim=300, pretrainedVectors='models/pretrained_vectors.txt')

In [ ]:
#model.save_model('fasttext_model.h')

In [8]:
model = fastText.load_model("models/fasttext_model.h")

In [9]:
words_ = open('tmp.txt').readline()
words_ = words_.split(" ")
words_ = np.unique(words_)
words_ = words_[1:]

In [10]:
X = [model.get_word_vector(word) for word in words_]

In [11]:
word2X = dict(zip(words_, X))

In [12]:
kNN = NearestNeighbors(n_neighbors=5, metric='euclidean').fit(X)

In [13]:
# нахождение 'синонимов'

def find_syn(word):
    w = morph.parse(word)[0].normal_form
    try:
        ind = kNN.kneighbors([word2X[w]])[1]
    except KeyError:
        print("Данного слова нет в словаре")
        return
    for w_ in words_[ind].ravel():
        print(w_)

In [14]:
find_syn('транспорт')

транспорт
транспортный
транспортировка
автотранспортный
трансплантация


**Классификация глаголов**

In [29]:
verbs = []

for word in words_:
    if morph.parse(word)[0].tag.cyr_repr.split(',')[0] == "ИНФ":
        verbs.append(word)
        print(word)

обвиняться
спрятаться
подсматривать
купаться
признать
усмехнуться
сделать
сказать
делать
признавать
рассказать
быть
выйти
прошататься
выйти
чувствовать
выбрать
сесть
вынуть
стать
закусывать
оборачиваться
купаться
делать
завтракать
заметить
стать
смотреть
завтракать
искупать
сказать
быть
смотреть
быть
быть
смягчить
быть
быть
оторвать
оживиться
зажестикулировать
представить
быть
подчёркивать
выделять
закашляться
возразить
рассказывать
сиять
быть
найти
быть
требоваться
уверять
быть
слушать
полузакрыть
очнуться
сделать
нахмуриться
сказать
быть
сказать
наклоняться
видеть
сказать
заключить
достигнуть
быть
засмеяться
звучать
послышаться
замолчать
закричать
рассказывать
знать
спасать
заслужить
ступать
повернуться
пойти
остановить
записывать
находиться
перейти
пройти
повалить
идти
сказать
подмигнуть
раскланяться
раскачиваться
исчезнуть


In [30]:
dict_words = dict(zip(verbs, np.zeros(len(verbs), dtype=int)))

In [31]:
strong_verbs = []

In [32]:
with open('verbs.txt', 'r', encoding="utf-8") as f:
    for verb in f.readlines():
        strong_verbs.append(verb[:-1])
        if verb[:-1] in dict_words.keys():
            dict_words[verb[:-1]] = 1

In [33]:
forms_of_verbs = {}

In [34]:
s = "".join(texts)
s = re.sub(r"[^A-Za-zА-Яа-я]+", " ", s)
words = [word.lower() for word in s.split(" ")]   
words_ = []
tag = set()
for word in tqdm.tqdm_notebook(words):
    normal_form = morph.parse(word)[0].normal_form
    if normal_form in strong_verbs:
        if normal_form in forms_of_verbs.keys():
            forms_of_verbs[normal_form].add(word)
        else:
            forms_of_verbs[normal_form] = set([word])    
        tag.add(morph.parse(word)[0].tag)

Посмотрим соотношений глаголов полномочий к остальным в обучающей выборке

In [235]:
np.unique(list(dict_words.values()), return_counts=True)

(array([0, 1]), array([225, 119], dtype=int64))

In [236]:
X = [model.get_word_vector(word) for word in verbs]

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, list(dict_words.values()), test_size=0.2)

**Logistic Regression**

In [41]:
parameters = {'penalty':('l1', 'l2'), 'C':[0.01, 1, 10, 100, 1000, 10000, 1e5, 1e6, 1e7, 1e8, 1e9],
             'max_iter':(100, 1000, 10000, 1e5)}
lr = LogisticRegression()
clf = GridSearchCV(lr, parameters, cv=5, scoring='f1', verbose=2)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 88 candidates, totalling 440 fits
[CV] C=0.01, max_iter=100, penalty=l1 ................................
[CV] ................. C=0.01, max_iter=100, penalty=l1, total=   0.0s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] C=0.01, max_iter=100, penalty=l1 ................................
[CV] ................. C=0.01, max_iter=100, penalty=l1, total=   0.0s
[CV] C=0.01, max_iter=100, penalty=l1 ................................
[CV] ................. C=0.01, max_iter=100, penalty=l1, total=   0.0s
[CV] C=0.01, max_iter=100, penalty=l1 ................................
[CV] ................. C=0.01, max_iter=100, penalty=l1, total=   0.0s
[CV] C=0.01, max_iter=100, penalty=l1 ................................
[CV] ................. C=0.01, max_iter=100, penalty=l1, total=   0.0s
[CV] C=0.01, max_iter=100, penalty=l2 ................................
[CV] ................. C=0.01, max_iter=100, penalty=l2, total=   0.0s
[CV] C=0.01, max_iter=100, penalty=l2 ................................
[CV] ................. C=0.01, max_iter=100, penalty=l2, total=   0.0s
[CV] C=0.01, max_iter=100, penalty=l2 ................................
[CV] ................. C=0.01, max_iter=100, penalty=l2, total=   0.0s
[CV] C

[CV] C=10, max_iter=100000.0, penalty=l2 .............................
[CV] .............. C=10, max_iter=100000.0, penalty=l2, total=   0.0s
[CV] C=10, max_iter=100000.0, penalty=l2 .............................
[CV] .............. C=10, max_iter=100000.0, penalty=l2, total=   0.0s
[CV] C=10, max_iter=100000.0, penalty=l2 .............................
[CV] .............. C=10, max_iter=100000.0, penalty=l2, total=   0.0s
[CV] C=10, max_iter=100000.0, penalty=l2 .............................
[CV] .............. C=10, max_iter=100000.0, penalty=l2, total=   0.0s
[CV] C=100, max_iter=100, penalty=l1 .................................
[CV] .................. C=100, max_iter=100, penalty=l1, total=   0.0s
[CV] C=100, max_iter=100, penalty=l1 .................................
[CV] .................. C=100, max_iter=100, penalty=l1, total=   0.0s
[CV] C=100, max_iter=100, penalty=l1 .................................
[CV] .................. C=100, max_iter=100, penalty=l1, total=   0.0s
[CV] C

[CV] C=10000, max_iter=100000.0, penalty=l1 ..........................
[CV] ........... C=10000, max_iter=100000.0, penalty=l1, total=   0.8s
[CV] C=10000, max_iter=100000.0, penalty=l1 ..........................
[CV] ........... C=10000, max_iter=100000.0, penalty=l1, total=   0.6s
[CV] C=10000, max_iter=100000.0, penalty=l1 ..........................
[CV] ........... C=10000, max_iter=100000.0, penalty=l1, total=   0.8s
[CV] C=10000, max_iter=100000.0, penalty=l1 ..........................
[CV] ........... C=10000, max_iter=100000.0, penalty=l1, total=   0.7s
[CV] C=10000, max_iter=100000.0, penalty=l2 ..........................
[CV] ........... C=10000, max_iter=100000.0, penalty=l2, total=   0.0s
[CV] C=10000, max_iter=100000.0, penalty=l2 ..........................
[CV] ........... C=10000, max_iter=100000.0, penalty=l2, total=   0.0s
[CV] C=10000, max_iter=100000.0, penalty=l2 ..........................
[CV] ........... C=10000, max_iter=100000.0, penalty=l2, total=   0.0s
[CV] C

[CV] C=10000000.0, max_iter=10000, penalty=l2 ........................
[CV] ......... C=10000000.0, max_iter=10000, penalty=l2, total=   0.0s
[CV] C=10000000.0, max_iter=10000, penalty=l2 ........................
[CV] ......... C=10000000.0, max_iter=10000, penalty=l2, total=   0.0s
[CV] C=10000000.0, max_iter=10000, penalty=l2 ........................
[CV] ......... C=10000000.0, max_iter=10000, penalty=l2, total=   0.0s
[CV] C=10000000.0, max_iter=10000, penalty=l2 ........................
[CV] ......... C=10000000.0, max_iter=10000, penalty=l2, total=   0.0s
[CV] C=10000000.0, max_iter=100000.0, penalty=l1 .....................
[CV] ...... C=10000000.0, max_iter=100000.0, penalty=l1, total=   0.1s
[CV] C=10000000.0, max_iter=100000.0, penalty=l1 .....................
[CV] ...... C=10000000.0, max_iter=100000.0, penalty=l1, total=   0.1s
[CV] C=10000000.0, max_iter=100000.0, penalty=l1 .....................
[CV] ...... C=10000000.0, max_iter=100000.0, penalty=l1, total=   0.1s
[CV] C

[Parallel(n_jobs=1)]: Done 440 out of 440 | elapsed:  1.1min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ('l1', 'l2'), 'C': [0.01, 1, 10, 100, 1000, 10000, 100000.0, 1000000.0, 10000000.0, 100000000.0, 1000000000.0], 'max_iter': (100, 1000, 10000, 100000.0)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=2)

In [42]:
clf.best_params_, clf.best_score_

({'C': 10000, 'max_iter': 100, 'penalty': 'l1'}, 0.5843279755973255)

In [43]:
lr = LogisticRegression(C=clf.best_params_['C'], max_iter=clf.best_params_['max_iter'], penalty=clf.best_params_['penalty'])
lr.fit(X_train, y_train)

LogisticRegression(C=10000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
with open('models/lr.pkl', "rb") as f:
    lr = joblib.load(f)

**SVM**

In [50]:
parameters = {'kernel':('linear', 'rbf'), 'C':[0.001, 0.01, 1, 10, 100, 1000, 10000, 1e5, 1e6, 1e7, 1e8],
             'tol':[0.1, 0.01, 0.001]}
svc = SVC()
clf = GridSearchCV(svc, parameters, cv=5, scoring='f1', verbose=2)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 66 candidates, totalling 330 fits
[CV] C=0.001, kernel=linear, tol=0.1 .................................
[CV] .................. C=0.001, kernel=linear, tol=0.1, total=   0.0s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] C=0.001, kernel=linear, tol=0.1 .................................
[CV] .................. C=0.001, kernel=linear, tol=0.1, total=   0.0s
[CV] C=0.001, kernel=linear, tol=0.1 .................................
[CV] .................. C=0.001, kernel=linear, tol=0.1, total=   0.0s
[CV] C=0.001, kernel=linear, tol=0.1 .................................
[CV] .................. C=0.001, kernel=linear, tol=0.1, total=   0.0s
[CV] C=0.001, kernel=linear, tol=0.1 .................................
[CV] .................. C=0.001, kernel=linear, tol=0.1, total=   0.0s
[CV] C=0.001, kernel=linear, tol=0.01 ................................
[CV] ................. C=0.001, kernel=linear, tol=0.01, total=   0.0s
[CV] C=0.001, kernel=linear, tol=0.01 ................................
[CV] ................. C=0.001, kernel=linear, tol=0.01, total=   0.0s
[CV] C=0.001, kernel=linear, tol=0.01 ................................
[CV] ................. C=0.001, kernel=linear, tol=0.01, total=   0.0s
[CV] C

[CV] C=10, kernel=rbf, tol=0.001 .....................................
[CV] ...................... C=10, kernel=rbf, tol=0.001, total=   0.0s
[CV] C=10, kernel=rbf, tol=0.001 .....................................
[CV] ...................... C=10, kernel=rbf, tol=0.001, total=   0.0s
[CV] C=10, kernel=rbf, tol=0.001 .....................................
[CV] ...................... C=10, kernel=rbf, tol=0.001, total=   0.0s
[CV] C=10, kernel=rbf, tol=0.001 .....................................
[CV] ...................... C=10, kernel=rbf, tol=0.001, total=   0.0s
[CV] C=100, kernel=linear, tol=0.1 ...................................
[CV] .................... C=100, kernel=linear, tol=0.1, total=   0.0s
[CV] C=100, kernel=linear, tol=0.1 ...................................
[CV] .................... C=100, kernel=linear, tol=0.1, total=   0.0s
[CV] C=100, kernel=linear, tol=0.1 ...................................
[CV] .................... C=100, kernel=linear, tol=0.1, total=   0.0s
[CV] C

[CV] C=100000.0, kernel=rbf, tol=0.01 ................................
[CV] ................. C=100000.0, kernel=rbf, tol=0.01, total=   0.0s
[CV] C=100000.0, kernel=rbf, tol=0.01 ................................
[CV] ................. C=100000.0, kernel=rbf, tol=0.01, total=   0.0s
[CV] C=100000.0, kernel=rbf, tol=0.01 ................................
[CV] ................. C=100000.0, kernel=rbf, tol=0.01, total=   0.0s
[CV] C=100000.0, kernel=rbf, tol=0.01 ................................
[CV] ................. C=100000.0, kernel=rbf, tol=0.01, total=   0.0s
[CV] C=100000.0, kernel=rbf, tol=0.001 ...............................
[CV] ................ C=100000.0, kernel=rbf, tol=0.001, total=   0.0s
[CV] C=100000.0, kernel=rbf, tol=0.001 ...............................
[CV] ................ C=100000.0, kernel=rbf, tol=0.001, total=   0.0s
[CV] C=100000.0, kernel=rbf, tol=0.001 ...............................
[CV] ................ C=100000.0, kernel=rbf, tol=0.001, total=   0.0s
[CV] C

[Parallel(n_jobs=1)]: Done 330 out of 330 | elapsed:   27.0s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'kernel': ('linear', 'rbf'), 'C': [0.001, 0.01, 1, 10, 100, 1000, 10000, 100000.0, 1000000.0, 10000000.0, 100000000.0], 'tol': [0.1, 0.01, 0.001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=2)

In [51]:
clf.best_params_, clf.best_score_

({'C': 1000000.0, 'kernel': 'linear', 'tol': 0.1}, 0.5605989575834366)

In [52]:
svc = SVC(C=clf.best_params_['C'], kernel=clf.best_params_['kernel'], tol=clf.best_params_['tol'])
svc.fit(X_train, y_train)

SVC(C=1000000.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.1, verbose=False)

In [74]:
with open('models/svc.pkl', "rb") as f:
    svc = joblib.load(f)

**Тестирование**

In [45]:
test_text = open(join(folder + '\\test\\Стальное колечко.txt')).readlines()

In [46]:
DICT_OF_VERBS = {}
words_ = process_text(test_text, dict_verbs=DICT_OF_VERBS)

In [47]:
verbs = []
for verb in list(DICT_OF_VERBS.keys()):
    verbs += DICT_OF_VERBS[verb]

In [48]:
len(list(DICT_OF_VERBS.keys()))

238

In [49]:
verbs = np.unique(verbs)
test = [model.get_word_vector(word) for word in list(DICT_OF_VERBS.keys())]
pred = lr.predict(test)
predicted_verbs = []
for i,verb in enumerate(list(DICT_OF_VERBS.keys())):
    if pred[i] == 1:
        predicted_verbs += DICT_OF_VERBS[verb]

**Записать результаты в файлик**

In [50]:
def print_result(dict_verbs, predicted_verbs, file='results/result_стальное колечко.txt'):
    f = open(file, mode='w')
    for verb in predicted_verbs:
        f.write(verb + '\n')

In [51]:
print_result(DICT_OF_VERBS, predicted_verbs)

In [54]:
len(predicted_verbs)

66

In [52]:
", ".join(predicted_verbs)

'говорит, говорил, кашлял, село, сказать, сказал, сказала, улетишь, налетел, залепил, утащило, продашь, курить, взял, возьму, возьми, верь, слыхала, повертывала, дай, попробую, заплакала, задымил, горюй, склевывал, вырвать, отгоняли, начинал, чирикать, шумел, шумели, шумишь, поймала, торчал, посадила, завязли, плакала, спрашивал, отвечала, норовит, открыла, выглянула, дунул, растрепал, шуршал, булькать, наломало, зацвели, отгребать, присела, схватила, издали, выскочила, затворить, наготовлю, затопим, вылечило, приходила, кивали, слушала, улыбалась, обдало, знала, охватишь, повидать, успею'

In [61]:
(238-66)/238

0.7226890756302521

In [60]:
k = 0
for verb in predicted_verbs:
    if morph.parse(verb)[0].normal_form not in strong_verbs:
        print(verb)
        k+=1

говорит
говорил
кашлял
село
сказать
сказал
сказала
улетишь
налетел
залепил
утащило
продашь
курить
взял
возьму
возьми
верь
слыхала
повертывала
попробую
заплакала
задымил
горюй
склевывал
вырвать
отгоняли
начинал
чирикать
шумел
шумели
шумишь
поймала
торчал
посадила
завязли
плакала
спрашивал
норовит
выглянула
дунул
растрепал
шуршал
булькать
наломало
зацвели
отгребать
присела
схватила
выскочила
затворить
наготовлю
затопим
вылечило
приходила
кивали
слушала
улыбалась
обдало
знала
охватишь
повидать
успею


In [55]:
v = []

In [56]:
with open('test_verbs.txt') as f:
    for verb in f.readlines():
        if verb.split("\t")[1][-1] == '\n':
            v.append(verb.split("\t")[1][:-1])

In [57]:
v

['быть',
 'мочь',
 'сказать',
 'говорить',
 'хотеть',
 'иметь',
 'видеть',
 'думать',
 'сделать',
 'понимать',
 'жить',
 'делать',
 'работать',
 'стоять',
 'пойти',
 'считать',
 'смотреть',
 'взять',
 'получать',
 'сидеть',
 'становиться',
 'спросить',
 'являться',
 'принимать',
 'любить',
 'оказаться',
 'оставаться',
 'увидеть',
 'прийти',
 'представлять',
 'писать',
 'казаться',
 'называть',
 'начинать',
 'понять',
 'дать',
 'начать',
 'остаться',
 'помнить',
 'находить',
 'ждать',
 'выйти',
 'следовать',
 'проводить',
 'происходить',
 'написать',
 'создавать',
 'посмотреть',
 'играть',
 'подумать',
 'решить',
 'вернуться',
 'проходить',
 'приходить',
 'ходить',
 'ответить',
 'отвечать',
 'лежать',
 'бывать',
 'читать',
 'использовать',
 'выходить',
 'рассказывать',
 'пройти',
 'занимать',
 'хотеться',
 'существовать',
 'получить',
 'продолжать',
 'показывать',
 'слышать',
 'стоить',
 'уходить',
 'прийтись',
 'вести',
 'смочь',
 'бояться',
 'спрашивать',
 'появиться',
 'находиться',


In [58]:
x = []
for verb in v:
    x.append(model.get_word_vector(verb))

In [60]:
pred = lr.predict(x)

In [62]:
np.unique(pred, return_counts=True)

(array([0, 1]), array([663, 335], dtype=int64))

In [63]:
for i in range(len(pred)):
    if pred[i] == 1:
        print(v[i])

сказать
говорить
иметь
понимать
стоять
взять
принимать
представлять
называть
начинать
понять
дать
начать
находить
следовать
проводить
написать
создавать
решить
приходить
ходить
отвечать
бывать
использовать
выходить
рассказывать
существовать
получить
уходить
вести
спрашивать
составлять
позволять
слушать
решать
вызывать
входить
держать
искать
требовать
определять
вспоминать
верить
открывать
устанавливать
возникать
узнать
выполнять
выступать
снимать
отмечать
успеть
уметь
рассматривать
рассказать
действовать
обеспечивать
подходить
вспомнить
состоять
поднимать
кричать
забывать
утверждать
сесть
узнавать
встречать
молчать
приезжать
открыть
расти
осуществлять
передавать
напоминать
судить
нести
участвовать
убивать
попадать
поддерживать
улыбаться
производить
бросать
являть
платить
содержать
устраивать
наблюдать
обладать
вставать
признавать
учитывать
разрабатывать
хватить
попробовать
мешать
улыбнуться
плакать
менять
рассчитывать
описывать
ездить
совершать
готовить
защищать
провести
выпускать
оказ

In [64]:
len(v)

998

In [65]:
663/998

0.6643286573146293